In [2]:
from bs4 import BeautifulSoup
import requests
import sys
import io
import tempfile
import time

def get_stock_dividend(no):
    url ='https://tw.stock.yahoo.com/d/s/dividend_' + str(no) +'.html'
    req = requests.get(url, timeout=5, allow_redirects=True)
    tmpName = tempfile.gettempdir() + '\\t2.html'
    open(tmpName, 'wb').write(req.content)
    results=[]
    with io.open(tmpName,'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr', {'bgcolor':'#FFFFFF'})
        for tr in trs:
            tds = tr.find_all('td')
            result=[]
            for td in tds:
                result.append(float(td.text))
            results.append(result)
    return results

def get_stock_price(no):
    url ='https://tw.stock.yahoo.com/q/ts?s='+str(no)
    req = requests.get(url, timeout=5, allow_redirects=True)
    tmpName = tempfile.gettempdir() + '\\t'+ no + '.html'
    open(tmpName,'wb').write(req.content)
    result=0.0
    with io.open(tmpName,'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr', {'bgcolor':'#ffffff'})
        tds = trs[0].find_all('td')
        if len(tds) > 3 :
            result= float(tds[3].text)
    return result;

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import sys

def auth_gss_client(path, scopes):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(path, scopes)
    return gspread.authorize(credentials)

def get_sheet_value(sheet, row, col):
    time.sleep(1.1)
    return sheet.cell(row,col).value

def set_sheet_value(sheet, row, col, value):
    time.sleep(1.1)
    sheet.update_cell(row,col,value)

def set_sheet_values(sheet, row, col, values):
    alpha='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    preA=alpha[col-1]+str(row)
    m=col+len(values)-2
    j=m/len(alpha)
    k=m%len(alpha)
    if j>0:
        preB=alpha[j-1]+alpha[k]+str(row)
    else:
        preB=alpha[col+len(values)-2]+str(row)
    label=preA+':'+preB
    cell_list=sheet.range(label)
    for c,v in zip(cell_list, values):
        c.value = v
    time.sleep(1.1)
    sheet.update_cells(cell_list)

def update_divident(no,rows):
    results = get_stock_dividend(no)
    colIndex = 14
    cell_value = []
    for result in results:
        cell_value.append(result[1])
        cell_value.append(result[2])
    set_sheet_values(wks.sheet1,rows,colIndex, cell_value)

auth_json_path = 'auth.json'
gss_scopes = ['https://spreadsheets.google.com/feeds']

gss_client = auth_gss_client(auth_json_path, gss_scopes)
s_key=open('spreadsheet_key','r').read()

wks = gss_client.open_by_key(s_key)
stockNo = 1
lineIndex = 2
run = True
print('Get Price\n')
while run:
    stockNo = get_sheet_value(wks.sheet1, lineIndex ,1)
    if stockNo != '':
        sys.stdout.write(str(stockNo)+',')
        set_sheet_value(wks.sheet1, lineIndex, 3, get_stock_price(stockNo))
        divValue = get_sheet_value(wks.sheet1, lineIndex, 14)
        if divValue == '':
            update_divident(stockNo, lineIndex)
        time.sleep(1)
        lineIndex += 1
    else:
        print('Bye!\n')
        run = False

Get Price

1101,1210,1215,1216,1227,1229,1231,1232,1234,1307,1513,1712,1717,1773,2105,2108,2317,2330,2412,2596,2616,2701,2801,2812,2820,2834,2845,2880,2881,2882,2883,2884,2885,2886,2887,2888,2889,2890,2891,2892,2912,4711,5438,5880,6136,6151,6183,6189,6216,6281,6803,8926,9905,9927,9939,3045,4904,5530,5519,2838,Bye!



In [10]:
from bs4 import BeautifulSoup
import requests
import io
import tempfile

def get_stock_priceDebug(no):
    tmpName = tempfile.gettempdir() + '\\t'+ no + '.html'
    result=0.0
    with io.open(tmpName,'r') as f:
        content = f.read()
        soup = BeautifulSoup(content, 'lxml')
        trs = soup.find_all('tr', {'bgcolor':'#ffffff'})
        tds = trs[0].find_all('td')
        print(trs)
        if len(tds) > 3 :
            result= float(tds[3].text)
    return result;

print(get_stock_priceDebug('2317'))

[<tr align="center" bgcolor="#ffffff"><td colspan="6"><br/>\u672c\u65e5\u5c1a\u7121\u6210\u4ea4\u660e\u7d30<br/><br/></td></tr>, <tr align="center" bgcolor="#ffffff"><td><br/>\u672c\u65e5\u5c1a\u7121\u91cf\u50f9\u8b8a\u5316\u5716<br/><br/></td></tr>]
0.0


sheetfu

In [16]:
from sheetfu import SpreadsheetApp

spreadsheet_key = open('spreadsheet_dbg_key','r').read()
spreadsheet = SpreadsheetApp('auth.json').open_by_id(spreadsheet_key)
sheet = spreadsheet.get_sheet_by_name('Sheet1')
data_range = sheet.get_data_range()

values = data_range.get_values()
#backgrounds = data_range.get_backgrounds()

print(data_range)
print(values)

<Range object A1:I3>
[[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [666, 2, 3, 4, 5, 6, 7, 8, 9]]
